# LoRA权重合并脚本

## 功能说明

本脚本用于将训练好的LoRA适配器权重与基础模型合并，生成完整的微调模型。

### 主要步骤
1. 加载基础模型和tokenizer
2. 加载LoRA适配器权重
3. 合并权重
4. 保存合并后的完整模型
5. 推理测试验证效果

### 版本信息
- mindnlp: 0.5.1
- mindspore: 2.7.0
- transformers: ~4.40-4.45（推荐）

### 适用场景
- 将LoRA权重永久合并到基础模型
- 部署时不依赖LoRA适配器
- 简化推理流程

## 1. 导入必要的库

In [ ]:
# 核心框架
import mindnlp
import mindspore
from mindnlp import core

# 模型相关
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# 查看版本信息
print(f"mindnlp版本: {mindnlp.__version__}")
print(f"mindspore版本: {mindspore.__version__}")

## 2. 配置路径

### 路径说明
- `base_model_name`: 基础模型名称（从Hugging Face加载）
- `lora_path`: 训练好的LoRA checkpoint路径
- `merged_path`: 合并后模型的保存路径

In [ ]:
# 基础模型名称
base_model_name = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'

# 训练好的 LoRA checkpoint 路径（修改为你的实际checkpoint路径）
# 示例：如果你的最佳checkpoint是checkpoint-1380，则设置为：
lora_path = "/home/ma-user/work/output/checkpoint-1380"

# 合并后模型的保存目录
merged_path = "/home/ma-user/work/merged_model"

print(f"基础模型: {base_model_name}")
print(f"LoRA权重路径: {lora_path}")
print(f"合并后保存路径: {merged_path}")

## 3. 加载基础模型和Tokenizer

In [ ]:
# 加载tokenizer
print("正在加载tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    base_model_name, 
    use_fast=False, 
    trust_remote_code=True
)
print("Tokenizer加载完成！")
tokenizer

In [ ]:
# 加载基础模型
print("正在加载基础模型，请稍候...")
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    ms_dtype=mindspore.bfloat16,  # 使用bfloat16数据类型
    device_map=0  # 指定设备
)

print("基础模型加载完成！")
print(f"模型参数量: {model.num_parameters():,}")

## 4. 加载LoRA权重

In [ ]:
# 加载 LoRA 适配器权重
print("正在加载LoRA适配器权重...")
model = PeftModel.from_pretrained(model, lora_path)
print("LoRA权重加载完成！")

## 5. 合并权重

使用 `merge_and_unload()` 方法将LoRA权重合并到基础模型中，生成完整的微调模型。

In [ ]:
# 合并 LoRA 权重到基础模型
print("正在合并权重...")
model = model.merge_and_unload()
print("权重合并完成！")

## 6. 保存合并后的模型

In [ ]:
# 保存完整的微调模型
print(f"正在保存模型到 {merged_path}...")
model.save_pretrained(merged_path)
tokenizer.save_pretrained(merged_path)

print("="*60)
print(f"✅ LoRA 权重已成功合并！")
print(f"✅ 合并后的模型保存在: {merged_path}")
print("="*60)

## 7. 推理测试

测试合并后的模型是否正常工作。

In [ ]:
# 将模型移至NPU设备
print("正在将模型移至NPU设备...")
model = model.npu()
print("模型已就绪！")

In [ ]:
# 测试样例
test_prompt = "月亮又圆又亮，所以古人称之为玉盘。"

print("="*60)
print("推理测试")
print("="*60)
print(f"输入文本: {test_prompt}")
print("-"*60)

# 构建对话输入
inputs = tokenizer.apply_chat_template(
    [
        {"role": "system", "content": "你是PDTB文本关系分析助手"},
        {"role": "user", "content": test_prompt}
    ],
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="ms",
    return_dict=True
).to('cuda')

# 生成配置
gen_kwargs = {
    "max_length": 2500,
    "do_sample": True,
    "top_k": 1
}

# 生成回答
with core.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    # 只保留生成的部分（去除输入）
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
print("模型输出:")
print(response)
print("="*60)

## 8. 更多测试样例（可选）

您可以在下方添加更多测试样例来验证模型效果。

In [ ]:
# 定义测试函数，方便测试多个样例
def test_model(prompt):
    """
    测试模型推理
    
    参数:
        prompt: 输入文本
    """
    print("="*60)
    print(f"输入: {prompt}")
    print("-"*60)
    
    inputs = tokenizer.apply_chat_template(
        [
            {"role": "system", "content": "你是PDTB文本关系分析助手"},
            {"role": "user", "content": prompt}
        ],
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="ms",
        return_dict=True
    ).to('cuda')
    
    gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
    
    with core.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    print("输出:")
    print(response)
    print("="*60)
    print()

# 测试样例列表
test_cases = [
    "他的有没有什么不足之处？我觉得他可以就是加一些他自己的感受，因为他如果光只说那些一系列的动作，就感觉很空白，没有什么情感在里面。",
    "星汉是什么？银河。",
    "对于花来说没有人欣赏是多么的悲惨，就像我们姑娘把自己打扮得花枝招展，却没有人欣赏一样是一种不幸"
]

# 执行测试
for test_case in test_cases:
    test_model(test_case)

## 9. 总结

### 完成的工作
1. ✅ 加载基础模型和LoRA适配器
2. ✅ 合并LoRA权重到基础模型
3. ✅ 保存完整的微调模型
4. ✅ 推理测试验证效果

### 模型使用
合并后的模型可以直接用于推理，无需再加载LoRA适配器：

```python
from transformers import AutoTokenizer, AutoModelForCausalLM

# 直接加载合并后的模型
tokenizer = AutoTokenizer.from_pretrained(merged_path)
model = AutoModelForCausalLM.from_pretrained(merged_path)
```

### 下一步
- 可以将合并后的模型部署到生产环境
- 或者继续在验证集上评估模型性能